In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=UZFHHWfCz2q-dJSAOy0aeBqGeADqv9SBYfsLS9Xcqks&tc=L2-7R7P4q7fwjEmwJhuhv-NfrKoxnYiUNXMvyxBMTFY&cc=8AGi6QQgqpNcH17ftVN7Oy7BvHVEvXSsok23j_gMJd0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXn49W8DbULL0ybalJBLGuVe13tfzMSbCwDswS41v7GqQ0TZCymBFh8

Successfully saved authorization token.


In [3]:
#Define  a region of interest
region = ee.Geometry.Rectangle(18.365219, -34.009378, 18.731303, -33.724338)

In [4]:
#Load Landsat data
image = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
.filterDate('2019-01-01', '2019-12-31')\
.filterBounds(region)\
.sort('CLOUD_COVER')\
.first()

In [5]:
#Make the training data
training =image.sample(**{
    'region': region,
    'scale': 30,
    'numPixels': 5000
})

In [6]:
# Initiate the clusterer and training it.
clusterer  = ee.Clusterer.wekaKMeans(15).train(training)

#Cluster the input using the trained clusterer
result = image.cluster(clusterer)

In [7]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[-33.85, 18.5], zoom_start=10)

# Add the layer to the map object.

my_map.add_ee_layer(result.clip(region).randomVisualizer(), {}, 'clusters')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)